# [Marias](Maria.md) prediction after one month use
ID: tnK534JMwwfhvUEycn69HPbhqkt2
Marias data is presented below after one week of use. The predictions reflects her persona and travel patterns, and explores preditions using Logistic regression.

In [1]:
import shutil
import sys, getopt
import tensorflow as tf
import time
import numpy as np
import pandas as pd
import os

print("Tensorflow version: "+tf.VERSION+ " Python version: "+sys.version)
TABLE = 'tnK534JMwwfhvUEycn69HPbhqkt2'
ROOT_DIR = "pendlaren"
export_dir = os.path.join(ROOT_DIR, TABLE)
print(os.path.abspath(export_dir))
tf.logging.set_verbosity(tf.logging.WARN)

def load_data(to_row,y_name='journey'):
    train = pd.read_csv("data/tnK534JMwwfhvUEycn69HPbhqkt2.csv",comment="-",skiprows=range(to_row,6762))
    train_x, train_y = train, train.pop(y_name)
    #I know WRONG
    test =pd.read_csv("data/tnK534JMwwfhvUEycn69HPbhqkt2.csv",comment="-",skiprows=range(1,6000))
    test_x, test_y = test, test.pop(y_name)
    train_y.astype(str)
    return (train_x, train_y.astype(str)), (test_x, test_y.astype(str))

def train_input_fn(features, labels, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
    return dataset

def eval_input_fn(features, labels, batch_size):
    features=dict(features)
    if labels is None:
        inputs = features
    else:
        inputs = (features, labels)
    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)
    return dataset

#print(my_feature_columns)
#Below LinearClassifier (Logistic regression) https://www.tensorflow.org/get_started/feature_columns
detectedActivity = tf.feature_column.categorical_column_with_identity(key="detectedActivity",num_buckets=7,default_value=0)
geoHash = tf.feature_column.categorical_column_with_hash_bucket("geoHash",100,dtype=tf.int64)
minuteOfDay = tf.feature_column.numeric_column("minuteOfDay")
weekDay = tf.feature_column.categorical_column_with_identity(key="weekday",num_buckets=7,default_value=0)
feature_columns = [detectedActivity,geoHash,minuteOfDay,weekDay]

retrainAll = True
if retrainAll:
    try:
        shutil.rmtree(export_dir)
        print("Retraining the model from scratch")
    except:
        print("Incremental training")
        
(train_x, train_y), (test_x, test_y)= load_data(to_row=500)
label_vocabulary = np.unique(train_y).tolist()
classifier = tf.estimator.LinearClassifier(feature_columns=feature_columns,
                                            n_classes=len(label_vocabulary),
                                            model_dir=export_dir,
                                            label_vocabulary=label_vocabulary)

classifier.train(input_fn=lambda:train_input_fn(train_x, train_y, 100), max_steps=100)

expected = ['8121680000', '8121680000', '8121680000']
predict_x = {
    'detectedActivity': [7,7,7],
    'geoHash': [1242479403, 124247900, 12424800],
    'minuteOfDay': [531,480,600],
    'weekday': [2,1,5],
}

predictions = classifier.predict(
    input_fn=lambda:eval_input_fn(predict_x,labels=None, batch_size=10))

template = ('Prediction is "{}" ({:.1f}%), expected "{}"')

for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    print(template.format(label_vocabulary[class_id],100 * probability, expec))

Tensorflow version: 1.8.0 Python version: 2.7.12 (default, Dec  4 2017, 14:50:18) 
[GCC 5.4.0 20160609]
/home/lars_rauer/notebookcommute/notebooks/commuter/pendlaren/tnK534JMwwfhvUEycn69HPbhqkt2
Retraining the model from scratch
Prediction is "8121680140" (100.0%), expected "8121680000"
Prediction is "8121680140" (99.4%), expected "8121680000"
Prediction is "8121680140" (99.2%), expected "8121680000"
